In [1]:
import numpy as np
import h5py
import pandas as pd
from scipy.linalg import sqrtm

#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, AveragePooling1D, Input, Flatten, Dropout, LeakyReLU
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L2
import tensorflow.keras.backend as K
import tensorflow as tf

import gc

import optuna

#tf.random.set_seed(59843)
#np.random.seed(59843)

tf.random.set_seed(42)
np.random.seed(42)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2022-10-11 11:54:57.176588: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-11 11:54:59.376638: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-10-11 11:54:59.412968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 11:54:59.413697: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 245.91GiB/s
2022-10-11 11:54:59.413735: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-11 11:54:59.437862: I tensorflow/stream_executor/pla

# Load data

In [3]:
def load_exps_and_labels(detections):
    all_exps, all_labels = [], []

    counter = 0

    for exp_name, data in detections.groupby('Exp'):
        counter += 1

        try:
            file = h5py.File(f'../data/real/{exp_name}.h5', 'r')
        except:
            print(f'Couldnt load exp {exp_name}')
            continue

        exp = []
        
        for channel_id in range(1, 17):
            channel = file[f'Chn{channel_id}']
            exp.append(channel)
        
        exp = np.swapaxes(exp, 1, 0)

        chirps_ids = []

        for _, (frame_id, depth, azi) in data[['Frame', 'Depth', 'Azi']].iterrows():
            chirps_ids = chirps_ids + list(np.linspace(frame_id*127, frame_id*128+127, 128, dtype=np.uint16))
            all_labels = all_labels + list(np.tile([depth], (128, 1)))
            #chirps_ids.append(int(frame_id*127 + 63))
            #all_labels.append(depth)

        exp = exp[chirps_ids]

        all_exps = all_exps + list(exp)

        #if counter > 2:
        #    break
    
    all_exps = np.array(all_exps)
    all_labels = np.array(all_labels)
    #all_exps = np.swapaxes(all_exps, 1, 0)
    all_exps[:, :, :50] = 0
    
    return all_exps.astype(np.float32), all_labels.astype(np.float32)
        

In [4]:
all_detections = pd.read_csv('../data/real/TABULAR_CSV_ALL_CLASSES.csv')

In [5]:
detections = all_detections[all_detections['Exp'].str.contains('EXP_17_')].query('Class == "Motorcycle" and Depth <= 25')

In [6]:
exps_train = ['EXP_17_A', 'EXP_17_B', 'EXP_17_C', 'EXP_17_D', 'EXP_17_E', 'EXP_17_F', 'EXP_17_G', 'EXP_17_H', 'EXP_17_I']
#exps_train = ['EXP_17_B']

detections_train = detections.query('Exp in @exps_train')
np.unique(detections_train['Exp'])


array(['EXP_17_B', 'EXP_17_C', 'EXP_17_D', 'EXP_17_E', 'EXP_17_F',
       'EXP_17_G', 'EXP_17_H', 'EXP_17_I'], dtype=object)

In [7]:
exps_val = ['EXP_17_M', 'EXP_17_Q', 'EXP_17_L']

detections_val = detections.query('Exp in @exps_val')
np.unique(detections_val['Exp'])


array(['EXP_17_L', 'EXP_17_M', 'EXP_17_Q'], dtype=object)

In [8]:
exps_val2 = ['EXP_17_N', 'EXP_17_P', 'EXP_17_R']

detections_val2 = detections.query('Exp in @exps_val2')
np.unique(detections_val2['Exp'])


array(['EXP_17_P', 'EXP_17_R'], dtype=object)

In [9]:
X_train, y_train = load_exps_and_labels(detections_train)
X_val, y_val = load_exps_and_labels(detections_val)
X_val2, y_val2 = load_exps_and_labels(detections_val2)

In [10]:
X_train.shape

(48256, 16, 1024)

In [11]:
X_val.shape

(12416, 16, 1024)

In [12]:
X_val2.shape

(15616, 16, 1024)

In [13]:
y_val.shape

(12416, 1)

# Preprocess data

## Swap axis

In [14]:
X_train = np.swapaxes(X_train, 2, 1)
X_val = np.swapaxes(X_val, 2, 1)
X_val2 = np.swapaxes(X_val2, 2, 1)

## Scale

In [15]:
#X_mean = 183.12559509277344
#X_std = 973.3739013671875

# All
X_mean = 185.10327
X_std = 979.17725

In [16]:
X_train = np.subtract(X_train, X_mean, out=X_train)
X_train = np.divide(X_train, X_std, out=X_train)

In [17]:
X_val = np.subtract(X_val, X_mean, out=X_val)
X_val = np.divide(X_val, X_std, out=X_val)

In [18]:
X_val2 = np.subtract(X_val2, X_mean, out=X_val2)
X_val2 = np.divide(X_val2, X_std, out=X_val2)

## Shuffle

In [19]:
random_ids = np.arange(len(X_train))
np.random.shuffle(random_ids)

X_train[:, :, :] = X_train[random_ids]
y_train[:, :] = y_train[random_ids]

# Build Model

In [20]:
def build_model(trial):
    model = Sequential([
        Input((1024, 16)),

        Conv1D(
            filters=32,
            kernel_regularizer=L2(trial.suggest_float("conv1_lambda", 0.01, 0.5)),
            kernel_size=3, 
            activation='relu'
        ),
        AveragePooling1D(
            strides=4
        ),

        Conv1D(
            filters=64, 
            kernel_regularizer=L2(trial.suggest_float("conv2_lambda", 0.01, 0.5)),
            kernel_size=3, 
            activation='relu'
        ),
        AveragePooling1D(
            strides=4
        ),

        Conv1D(
            filters=128, 
            kernel_regularizer=L2(trial.suggest_float("conv3_lambda", 0.01, 0.5)),
            kernel_size=3, 
            activation='relu'
        ),
        AveragePooling1D(
            strides=4
        ),

        Flatten(),
        
        Dense(
            units=1,
            activation='linear',
            kernel_regularizer=L2(trial.suggest_float("dense_lambda", 0.01, 0.5)))
    ])

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mean_absolute_error'])

    return model

# Train Model

In [20]:
study = optuna.create_study(direction='minimize')

def objective(trial):
    
    model = build_model(trial)

    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=trial.suggest_int('epochs', 1, 15),
              batch_size=128)

    val = model.evaluate(X_val, y_val)[1]

    del model
    K.clear_session()

    return val

study.optimize(objective, n_trials=100, gc_after_trial=True)

[I 2022-08-29 11:50:32,098] A new study created in memory with name: no-name-f45ce251-b779-49f2-9813-ac5f11ff385f
2022-08-29 11:50:32.124113: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-29 11:50:32.158642: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2599990000 Hz
2022-08-29 11:50:32.159335: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55557a8cf4e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-29 11:50:32.159361: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-29 11:50:32.227534: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA nod

Epoch 1/6


2022-08-29 11:50:36.282610: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-08-29 11:50:36.626246: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2022-08-29 11:50:37.955137: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2022-08-29 11:50:38.066485: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-08-29 11:50:41.208964: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 3.70G (3969225728 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


377/377 [==============================] - ETA: 0s - loss: 59.4903 - mean_absolute_error: 5.4594

2022-08-29 11:50:48.434036: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 813694976 exceeds 10% of free system memory.


...

[I 2022-08-29 12:52:26,259] Trial 97 finished with value: 4.455135345458984 and parameters: {'conv1_lambda': 0.40992236004278637, 'conv2_lambda': 0.3204798512414752, 'conv3_lambda': 0.22702676376531833, 'dense_lambda': 0.13153015774105525, 'epochs': 10}. Best is trial 84 with value: 2.6249778270721436.


Epoch 1/9
377/377 [==============================] - 4s 11ms/step - loss: 59.4462 - mean_absolute_error: 5.4011 - val_loss: 37.0212 - val_mean_absolute_error: 4.3988
Epoch 2/9
377/377 [==============================] - 3s 9ms/step - loss: 36.2964 - mean_absolute_error: 4.3498 - val_loss: 26.2170 - val_mean_absolute_error: 3.4263
Epoch 3/9
377/377 [==============================] - 4s 9ms/step - loss: 30.0524 - mean_absolute_error: 3.8032 - val_loss: 27.0394 - val_mean_absolute_error: 3.4515
Epoch 4/9
377/377 [==============================] - 4s 9ms/step - loss: 27.1766 - mean_absolute_error: 3.4857 - val_loss: 23.2115 - val_mean_absolute_error: 3.0033
Epoch 5/9
377/377 [==============================] - 4s 9ms/step - loss: 25.9961 - mean_absolute_error: 3.3478 - val_loss: 23.2100 - val_mean_absolute_error: 3.0006
Epoch 6/9
377/377 [==============================] - 4s 9ms/step - loss: 24.9200 - mean_absolute_error: 3.2140 - val_loss: 26.8508 - val_mean_absolute_error: 3.2806
Epoch 7/9

[I 2022-08-29 12:53:03,169] Trial 98 finished with value: 2.8848717212677 and parameters: {'conv1_lambda': 0.34962975888642367, 'conv2_lambda': 0.26915638113371537, 'conv3_lambda': 0.41734500457065704, 'dense_lambda': 0.19464404561789256, 'epochs': 9}. Best is trial 84 with value: 2.6249778270721436.


Epoch 1/12
377/377 [==============================] - 4s 11ms/step - loss: 56.6664 - mean_absolute_error: 5.3856 - val_loss: 36.2920 - val_mean_absolute_error: 4.3370
Epoch 2/12
377/377 [==============================] - 4s 9ms/step - loss: 35.5671 - mean_absolute_error: 4.2947 - val_loss: 24.9375 - val_mean_absolute_error: 3.2019
Epoch 3/12
377/377 [==============================] - 4s 9ms/step - loss: 29.0233 - mean_absolute_error: 3.6910 - val_loss: 25.8090 - val_mean_absolute_error: 3.2980
Epoch 4/12
377/377 [==============================] - 4s 9ms/step - loss: 26.5628 - mean_absolute_error: 3.4487 - val_loss: 23.6383 - val_mean_absolute_error: 3.0528
Epoch 5/12
377/377 [==============================] - 4s 9ms/step - loss: 25.1397 - mean_absolute_error: 3.3008 - val_loss: 22.4280 - val_mean_absolute_error: 2.9436
Epoch 6/12
377/377 [==============================] - 4s 9ms/step - loss: 24.1674 - mean_absolute_error: 3.1886 - val_loss: 22.9356 - val_mean_absolute_error: 2.9771
Epo

[I 2022-08-29 12:53:50,851] Trial 99 finished with value: 2.86090087890625 and parameters: {'conv1_lambda': 0.38647991126188763, 'conv2_lambda': 0.2322246561499634, 'conv3_lambda': 0.289732585309104, 'dense_lambda': 0.2535446574226776, 'epochs': 12}. Best is trial 84 with value: 2.6249778270721436.


In [2]:
# {'conv1_lambda': 0.37217565639573935, 'conv2_lambda': 0.3703556673278788, 'conv3_lambda': 0.36107002903413576, 'dense_lambda': 0.13567068335576418, 'epochs': 9}

model = Sequential([
    Input((1024, 16)),

    Conv1D(
        filters=32,
        kernel_regularizer=L2(0.37),
        kernel_size=3, 
        activation='relu'
    ),
    AveragePooling1D(
        strides=4
    ),

    Conv1D(
        filters=64, 
        kernel_regularizer=L2(0.37),
        kernel_size=3, 
        activation='relu'
    ),
    AveragePooling1D(
        strides=4
    ),

    Conv1D(
        filters=128, 
        kernel_regularizer=L2(0.36),
        kernel_size=3, 
        activation='relu'
    ),
    AveragePooling1D(
        strides=4
    ),

    Flatten(),
    
    Dense(
        units=1,
        activation='linear',
        kernel_regularizer=L2(0.14))
])

model.compile(optimizer='adam',
                loss='mse',
                metrics=['mean_absolute_error'])

2022-10-11 11:55:10.143552: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-11 11:55:10.173289: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2599990000 Hz
2022-10-11 11:55:10.174205: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557a8aa83500 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-10-11 11:55:10.174252: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-10-11 11:55:10.428871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1022, 32)          1568      
_________________________________________________________________
average_pooling1d (AveragePo (None, 256, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 254, 64)           6208      
_________________________________________________________________
average_pooling1d_1 (Average (None, 64, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 62, 128)           24704     
_________________________________________________________________
average_pooling1d_2 (Average (None, 16, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

In [21]:
model.fit(X_train, y_train, validation_data=(X_val, y_val),
            epochs=9,
            batch_size=128)

2022-09-15 14:53:36.427422: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 3162505216 exceeds 10% of free system memory.


Epoch 1/9


2022-09-15 14:53:38.325837: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-09-15 14:53:38.676355: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2022-09-15 14:53:40.073527: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2022-09-15 14:53:40.232789: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-09-15 14:53:44.680149: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 3.61G (3877127936 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


373/377 [============================>.] - ETA: 0s - loss: 59.9180 - mean_absolute_error: 5.4671

2022-09-15 14:53:53.435799: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 813694976 exceeds 10% of free system memory.


377/377 [==============================] - 4s 11ms/step - loss: 59.7326 - mean_absolute_error: 5.4614 - val_loss: 48.1819 - val_mean_absolute_error: 5.5799
Epoch 2/9
377/377 [==============================] - 4s 10ms/step - loss: 38.3263 - mean_absolute_error: 4.6134 - val_loss: 28.6941 - val_mean_absolute_error: 3.6358
Epoch 3/9
377/377 [==============================] - 4s 10ms/step - loss: 30.9769 - mean_absolute_error: 3.9828 - val_loss: 25.1013 - val_mean_absolute_error: 3.3454
Epoch 4/9
377/377 [==============================] - 4s 10ms/step - loss: 27.3339 - mean_absolute_error: 3.5750 - val_loss: 22.2117 - val_mean_absolute_error: 2.9595
Epoch 5/9
377/377 [==============================] - 4s 10ms/step - loss: 25.7972 - mean_absolute_error: 3.3791 - val_loss: 21.9444 - val_mean_absolute_error: 2.9313
Epoch 6/9
377/377 [==============================] - 4s 10ms/step - loss: 24.4021 - mean_absolute_error: 3.2058 - val_loss: 21.3059 - val_mean_absolute_error: 2.8258
Epoch 7/9
377/

In [22]:
del X_train
del y_train
gc.collect()

1490

# FID Socore

In [23]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if np.iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
	
	return fid

In [24]:
model.pop()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1022, 32)          1568      
_________________________________________________________________
average_pooling1d (AveragePo (None, 256, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 254, 64)           6208      
_________________________________________________________________
average_pooling1d_1 (Average (None, 64, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 62, 128)           24704     
_________________________________________________________________
average_pooling1d_2 (Average (None, 16, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

In [25]:
X_val2.shape
#np.random.shuffle(X_val)

(15616, 1024, 16)

In [26]:
calculate_fid(model, X_val2[:10000], X_val[:10000])

2022-09-15 14:54:38.514490: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 655360000 exceeds 10% of free system memory.
2022-09-15 14:54:46.599753: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 655360000 exceeds 10% of free system memory.


4.950390282922341

In [27]:
noise = np.random.randn(10000, 1024, 16)
calculate_fid(model, X_val[:10000], noise)

2022-09-15 14:54:55.973355: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 655360000 exceeds 10% of free system memory.


54.91829309174981

In [28]:
generations = np.load('../data/generated/conditiona-nooutliers-2.npy')
generations = np.swapaxes(generations, 2, 1)

In [29]:
generations.shape

(6000, 1024, 16)

In [30]:
generations_scaled = np.subtract(generations, X_mean, out=generations)
generations_scaled = np.divide(generations_scaled, X_std, out=generations_scaled)
np.random.shuffle(generations_scaled)

In [31]:
calculate_fid(model, X_val2[:10000], generations_scaled[:10000])

5.567578344917186

In [32]:
exp_17_m = np.load('../data/preprocessed/EXP_17_M_chirps.npy')
exp_17_m = np.swapaxes(exp_17_m, 2, 1)

In [33]:
exp_17_m.shape

(6144, 1024, 16)

In [34]:
exp_17_m_scaled = np.subtract(exp_17_m, X_mean, out=exp_17_m)
exp_17_m_scaled = np.divide(exp_17_m_scaled, X_std, out=exp_17_m_scaled)
np.random.shuffle(exp_17_m_scaled)

In [35]:
calculate_fid(model, exp_17_m_scaled[:3000], exp_17_m_scaled[3000:6000])

0.05864832870604299

In [36]:
calculate_fid(model, exp_17_m_scaled[:3000], generations_scaled[:3000])

0.5068574309373406

In [37]:
exp_17_r = np.load('../data/preprocessed/EXP_17_R_chirps.npy')
exp_17_r = np.swapaxes(exp_17_r, 2, 1)

In [38]:
exp_17_r.shape

(9984, 1024, 16)

In [39]:
exp_17_r_scaled = np.subtract(exp_17_r, X_mean, out=exp_17_r)
exp_17_r_scaled = np.divide(exp_17_r_scaled, X_std, out=exp_17_r_scaled)
np.random.shuffle(exp_17_r_scaled)

In [40]:
calculate_fid(model, exp_17_r_scaled[:3000], exp_17_r_scaled[3000:6000])

0.010417899582522351

In [41]:
calculate_fid(model, exp_17_r_scaled[:3000], generations_scaled[:3000])

18.11957006281437

In [42]:
calculate_fid(model, exp_17_r_scaled[:300], exp_17_m_scaled[:3000])

18.183394780386713